Overview
*** Write an overview of the case study that you are working on. (MINIMUM 200 words) ***
1. When it comes to questions and answers humans have a different way of
understanding and answering the question asked. In this case study we are going
to understand the same with the help of (Question-Answer) dataset from top
websites.
2. Humans have a unique way of answering any question asked. We first understand
the question, the intuition behind it, the significance of the question etc. With
machine it is different.
3. In this problem, we will help the machine understand these aspects of question
with the help of a question-answer dataset from different websites.
4. In our dataset we have 30 different aspects of question-answer like
‘question_conversational’, ‘question_body_critical’, ‘answer_helpful’ etc.
5. Our goal will be to find the scores for each of these aspects with the help of our
training dataset. In our training dataset we have scores associated with each of
these labels in the range of [0,1]. We will make our model learn these aspects and
try to evaluate the same for our test dataset.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow.keras.backend as K
import gc
import os
from scipy.stats import spearmanr
from math import floor, ceil
import seaborn as sns

np.set_printoptions(suppress=True)

In [ ]:
path='/kaggle/input/google-quest-challenge/'

**Reading Our Datsets**

In [ ]:
#train dataset
train_df=pd.read_csv(path+'train.csv')
train_df.head()

In [ ]:
#test data
test_df=pd.read_csv(path+'test.csv')
test_df.head()

In [ ]:
#submission file
submission=pd.read_csv(path+'sample_submission.csv')
submission.head()

In [ ]:
#columns in our train_data
train_columns=train_df.columns
test_columns=test_df.columns
print('-'*15+'Train Columns'+'-'*15)
print(train_columns)
print('-'*15+'Train data Shape'+'-'*15)
print(train_df.shape)
print('-'*15+'Test Columns'+'-'*15)
print(test_columns)
print('-'*15+'Test data Shape'+'-'*15)
print(test_df.shape)

We thus have 6079 training points and 476 test points/ We have 41 train columns and 11 test columns. For the 30 columns we have to predict value for the test dataset. So, let us have a look at these 30 values

In [ ]:
target_values=train_columns[11:]
print('Target columns are: ')
target_values

For these 30 columns we need to find value for each point in the test dataset in the range of [0,1]

Now let us have a look at how these target_values are distributed across our training dataset

**Data Cleaning & Featurization**

Now as we have explored our data it is time to text data in our dataset and obtain new features depending on our dataset

**Remove Punctuations & Decontracted Words**

In [ ]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"couldn\'t", " could not", phrase)
    phrase = re.sub(r"couldnt", " could not", phrase)
    phrase = re.sub(r"didn\'t", " did not", phrase)
    phrase = re.sub(r"doesn\'t", " does not", phrase)
    phrase = re.sub(r"don\'t", " do not", phrase)
    phrase = re.sub(r"hadn\'t", " had not", phrase)
    phrase = re.sub(r"hasn\'t", " has not", phrase)
    phrase = re.sub(r"haven\'t", " have not", phrase)
    phrase = re.sub(r"he\'ll", " he will", phrase)
    phrase = re.sub(r"he\'d", " he would", phrase)
    phrase = re.sub(r"didn\'t", " did not", phrase)
    phrase = re.sub(r"wasn\'t", " was not", phrase)
    phrase = re.sub(r"you\'re", " you are", phrase)
    return phrase

In [ ]:
#code reference: https://www.kaggle.com/codename007/start-from-here-quest-complete-eda-fe


def clean_text(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = text.lower().split()   
    text = " ".join(text)
    return(text)

In [ ]:
#cleaning of train data
train_df['question_body']=train_df['question_body'].apply(clean_text)
train_df['question_body']=train_df['question_body'].apply(decontracted)
train_df['question_title']=train_df['question_body'].apply(clean_text)
train_df['question_title']=train_df['question_body'].apply(decontracted)
train_df['answer']=train_df['question_body'].apply(clean_text)
train_df['answer']=train_df['question_body'].apply(decontracted)

#cleaning of test data
test_df['question_body']=test_df['question_body'].apply(clean_text)
test_df['question_body']=test_df['question_body'].apply(decontracted)
test_df['question_title']=test_df['question_body'].apply(clean_text)
test_df['question_title']=test_df['question_body'].apply(decontracted)
test_df['answer']=test_df['question_body'].apply(clean_text)
test_df['answer']=test_df['question_body'].apply(decontracted)


**Printing Our Cleaned Data**

In [ ]:
train_df.head()

In [ ]:
test_df.head()

#Training with a Baseline Model

Splitting our train data into train and test dataset for our validating our model

In [ ]:
random.seed(10)
print(random.sample([1,2,3,4,5,6,7,8,9,10],5))
print(random.sample([1,2,3,4,5,6,7,8,9,10],3))

random.seed(10)
print(random.sample([1,2,3,4,5,6,7,8,9,10],5))
print(random.sample([1,2,3,4,5,6,7,8,9,10],3))

random.seed(10)
print(random.sample([1,2,3,4,5,6,7,8,9,10],5))
print(random.sample([1,2,3,4,5,6,7,8,9,10],3))


In [ ]:
import random

random.seed(42)
valid_n_test_indexes=random.sample(list(range(1,len(train_df))),int(len(train_df)*0.3))
train_indexes = list(set(list(range(1,len(train_df))))-set(valid_n_test_indexes))
valid_indexes=random.sample(valid_n_test_indexes,int(len(train_df)*0.2))
test_indexes=list(set(valid_n_test_indexes)-set(valid_indexes))
                        

In [ ]:
X_train=train_df.iloc[train_indexes]
X_valid=train_df.iloc[valid_indexes]
X_test=train_df.iloc[test_indexes]

print('Train data shape ='+str(X_train.shape))
print('Valid data shape ='+str(X_valid.shape))
print('Test data shape ='+str(X_test.shape))

In [ ]:
all_words=X_train['question_title']+X_train['question_body']+X_train['answer']

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(all_words)

In [ ]:
#for train data
quest_title = tokenizer.texts_to_sequences(X_train['question_title'])
quest_body = tokenizer.texts_to_sequences(X_train['question_body'])
answer = tokenizer.texts_to_sequences(X_train['answer'])

In [ ]:
#for valid data

quest_title_valid = tokenizer.texts_to_sequences(X_valid['question_title'])
quest_body_valid = tokenizer.texts_to_sequences(X_valid['question_body'])
answer_valid = tokenizer.texts_to_sequences(X_valid['answer'])

In [ ]:
#for test data taken from train dataset

quest_title_ts = tokenizer.texts_to_sequences(X_test['question_title'])
quest_body_ts = tokenizer.texts_to_sequences(X_test['question_body'])
answer_ts = tokenizer.texts_to_sequences(X_test['answer'])

In [ ]:
#for test data

quest_title_test = tokenizer.texts_to_sequences(test_df['question_title'])
quest_body_test = tokenizer.texts_to_sequences(test_df['question_body'])
answer_test = tokenizer.texts_to_sequences(test_df['answer'])

In [ ]:
title_lens=[len(i) for i in quest_title]
quest_lens=[len(i) for i in quest_body]
ans_lens=[len(i) for i in answer]

Now we will see the percentilte of lenghts to decide our max_tensor_len

In [ ]:
#len for question body
percentile_train_len = sorted(np.percentile((quest_lens),np.array(range(0,110,10))))
k=0
for i in range(len(percentile_train_len)):
  print(str(k)+' th percentile is '+str(percentile_train_len[i]))
  k+=10

In [ ]:
percentile_train_len = sorted(np.percentile((quest_lens),np.array(range(90,101))))
k=90
for i in range(len(percentile_train_len)):
  print(str(k)+' th percentile is '+str(percentile_train_len[i]))
  k+=1

In [ ]:
#percentiles for answers
percentile_train_len = sorted(np.percentile((ans_lens),np.array(range(0,110,10))))
k=0
for i in range(len(percentile_train_len)):
  print(str(k)+' th percentile is '+str(percentile_train_len[i]))
  k+=10

In [ ]:
percentile_train_len = sorted(np.percentile((ans_lens),np.array(range(91,101))))
k=0
for i in range(len(percentile_train_len)):
  print(str(k)+' th percentile is '+str(percentile_train_len[i]))
  k+=10

As we see that more than 99% values have lenght of question or answer not more than 750. We take it as our max len.

In [ ]:
#for train data

padded_question_title_train=np.array(tf.keras.preprocessing.sequence.pad_sequences(quest_title, maxlen=800, padding='post'))
padded_question_body_train=np.array(tf.keras.preprocessing.sequence.pad_sequences(quest_body, maxlen=800, padding='post'))
padded_question_answer_train=np.array(tf.keras.preprocessing.sequence.pad_sequences(answer, maxlen=800, padding='post'))

print('shape of question title: '+str(padded_question_title_train.shape))
print('shape of question body: '+str(padded_question_body_train.shape))
print('shape of answer: '+str(padded_question_answer_train.shape))

In [ ]:
#for valid data

padded_question_title_valid=np.array(tf.keras.preprocessing.sequence.pad_sequences(quest_title_valid, maxlen=800, padding='post'))
padded_question_body_valid=np.array(tf.keras.preprocessing.sequence.pad_sequences(quest_body_valid, maxlen=800, padding='post'))
padded_question_answer_valid=np.array(tf.keras.preprocessing.sequence.pad_sequences(answer_valid, maxlen=800, padding='post'))

print('shape of question title: '+str(padded_question_title_valid.shape))
print('shape of question body: '+str(padded_question_body_valid.shape))
print('shape of answer: '+str(padded_question_answer_valid.shape))

In [ ]:
#for test data taken from train_dataset

padded_question_title_ts=np.array(tf.keras.preprocessing.sequence.pad_sequences(quest_title_ts, maxlen=800, padding='post'))
padded_question_body_ts=np.array(tf.keras.preprocessing.sequence.pad_sequences(quest_body_ts, maxlen=800, padding='post'))
padded_question_answer_ts=np.array(tf.keras.preprocessing.sequence.pad_sequences(answer_ts, maxlen=800, padding='post'))

print('shape of question title: '+str(padded_question_title_ts.shape))
print('shape of question body: '+str(padded_question_body_ts.shape))
print('shape of answer: '+str(padded_question_answer_ts.shape))

In [ ]:
#for test data

padded_question_title_test=np.array(tf.keras.preprocessing.sequence.pad_sequences(quest_title_test, maxlen=800, padding='post'))
padded_question_body_test=np.array(tf.keras.preprocessing.sequence.pad_sequences(quest_body_test, maxlen=800, padding='post'))
padded_question_answer_test=np.array(tf.keras.preprocessing.sequence.pad_sequences(answer_test, maxlen=800, padding='post'))

print('shape of question title: '+str(padded_question_title_test.shape))
print('shape of question body: '+str(padded_question_body_test.shape))
print('shape of answer: '+str(padded_question_answer_test.shape))

In [ ]:
vocab_size=len(tokenizer.word_index)+1
print(vocab_size)

**Glove Vector Words**

In [ ]:
import pickle

with open(r'/kaggle/input/glove-vectors/glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = model.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
import tensorflow.keras.backend as T
from keras.callbacks import Callback
from scipy.stats import spearmanr

def compute_spearmanr(trues, preds):
    rhos = []
 
    for col_trues, col_pred in zip(trues.T, preds.T):
     
        rhos.append(
            spearmanr(col_trues, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.mean(rhos)


class CustomCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, valid_data, test_data, batch_size=16, fold=None):

        self.valid_inputs = valid_data[0]
        self.valid_outputs = valid_data[1]
        self.test_inputs = test_data
        
        self.batch_size = batch_size
        self.fold = fold
        
    def on_train_begin(self, logs={}):
        self.valid_predictions = []
        self.test_predictions = []
        
    def on_epoch_end(self, epoch, logs={}):
        print(len(self.valid_predictions))
        self.valid_predictions.append(
            self.model.predict(self.valid_inputs, batch_size=self.batch_size))
      
        rho_val = compute_spearmanr(
            self.valid_outputs, np.average(self.valid_predictions, axis=0))
        
        print("\nvalidation rho: %.4f" % rho_val)
        
        if self.fold is not None:
            self.model.save_weights(f'bert-base-{fold}-{epoch}.h5py')
        
        self.test_predictions.append(
            self.model.predict(self.test_inputs, batch_size=self.batch_size)
        )

In [ ]:
from tensorflow.keras.layers import Input,LSTM,Embedding,Dense,TimeDistributed,Concatenate



input_quest=Input(shape=(800,))
embedding_layer1=Embedding(vocab_size,output_dim=300,input_length=800)(input_quest)
lstm_q=LSTM(100, activation='tanh', recurrent_activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform',return_sequences = True)(embedding_layer1)
flatten_question=tf.keras.layers.Flatten()(lstm_q)


input_ans=Input(shape=(800,))
embedding_layer2=Embedding(vocab_size,output_dim=300, weights = [embedding_matrix],input_length=800)(input_ans)
lstm_a=LSTM(100, activation='tanh', recurrent_activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform',return_sequences = True)(embedding_layer2)
flatten_answer=tf.keras.layers.Flatten()(lstm_a)

concatenate_layer = tf.keras.layers.Concatenate(axis=1)(inputs = [flatten_question, flatten_answer])

dense_layer1 = Dense(512,activation='tanh',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(concatenate_layer)
dense_layer2 = Dense(256,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(dense_layer1)
dense_layer3 = Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(dense_layer2)
output =  tf.keras.layers.Dense(30,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(dense_layer3)
model = tf.keras.Model(inputs=[input_quest,input_ans],outputs=output)
model.summary()

In [ ]:
adam = tf.keras.optimizers.Adam(0.0003)
#checkpoint,earlystop,
model.compile(loss='mse', optimizer=adam)

In [ ]:
inputs_train=[padded_question_body_train,padded_question_answer_train]
inputs_valid=[padded_question_body_valid,padded_question_answer_valid]
inputs_test=[padded_question_body_test,padded_question_answer_test]
train_output=np.array(X_train[target_values])
valid_output=np.array(X_valid[target_values])


In [ ]:
  custom_callback = CustomCallback(
        valid_data=(inputs_valid,valid_output), 
        test_data=inputs_test,
        batch_size=16,
        fold=None)


In [ ]:
model.fit(inputs_train,train_output,batch_size=16,epochs =30,validation_data=(inputs_valid,valid_output),callbacks=[custom_callback])

In [ ]:
test_output=model.predict(inputs_test)

In [ ]:
print(len(test_output[:,0]))

for i in range(len(target_values)):
  submission[target_values[i]]=test_output[:,i]

In [ ]:
submission.shape

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
os.listdir(path)